# References

https://www.kaggle.com/code/mtszkw/technical-indicators-for-trading-stocks
https://www.kaggle.com/code/thebrownviking20/everything-you-can-do-with-a-time-series
https://www.diva-portal.org/smash/get/diva2:1775077/FULLTEXT01.pdf
https://pub.towardsai.net/predicting-stock-prices-using-arima-fourier-transforms-and-technical-indicators-with-deep-43a164859683


## Import dependencies

In [ ]:
! pip install -r requirements.txt

In [ ]:
import os
import pandas as pd
import gdown

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import nbformat
import plotly as py
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots

import numpy as np                                    # For matrices!
import matplotlib.pyplot as plt                       # To visualize
from sklearn.linear_model import LinearRegression     # For the regression itself
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.metrics import mean_squared_error, f1_score
from sklearn.preprocessing import OneHotEncoder       # To convert discrete strings to vectors!
from sklearn.preprocessing import normalize           # For normalizing
import seaborn as sns                                 # For plots

## Change default chart layout

In [ ]:
# Show charts when running kernel
init_notebook_mode(connected=True)

# Change default background color for all visualizations
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(250,250,250,0.8)')
fig = go.Figure(layout=layout)
templated_fig = pio.to_templated(fig)
pio.templates['my_template'] = templated_fig.layout.template
pio.templates.default = 'my_template'

## Download datasets

In [ ]:
# SnP500
gdown.download("https://docs.google.com/uc?id=1KveL-W2L6YxO-1NKn0n6FeGTKLWgvAL4", "ES_continuous_adjusted_1min_2011_2018.txt", quiet=False)

# Nasdaq
gdown.download("https://docs.google.com/uc?id=1NyHRoU2YiaCKIhy7afUTuAmC0haWK1Ny", "NQ_continuous_adjusted_1min_2011_2018.txt", quiet=False)

## Read datasets as CSV

In [ ]:
ES_1min_raw = pd.read_csv('ES_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
ES_5min_raw = pd.read_csv('ES_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
ES_30min_raw = pd.read_csv('ES_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
ES_1hour_raw = pd.read_csv('ES_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')

NQ_1min_raw = pd.read_csv('NQ_continuous_adjusted_1min_2011_2018.txt', encoding='latin-1')
NQ_5min_raw = pd.read_csv('NQ_continuous_adjusted_5min_2011_2018.txt', encoding='latin-1')
NQ_30min_raw = pd.read_csv('NQ_continuous_adjusted_30min_2011_2018.txt', encoding='latin-1')
NQ_1hour_raw = pd.read_csv('NQ_continuous_adjusted_1hour_2011_2018.txt', encoding='latin-1')

## Calc Moving Average Data


In [ ]:
# SnP500

ES_1min_raw['EMA_9'] = ES_1min_raw['Close'].ewm(5).mean().shift()
ES_1min_raw['SMA_50'] = ES_1min_raw['Close'].rolling(50).mean().shift()
ES_1min_raw['SMA_100'] = ES_1min_raw['Close'].rolling(100).mean().shift()
ES_1min_raw['SMA_200'] = ES_1min_raw['Close'].rolling(200).mean().shift()

ES_5min_raw['EMA_9'] = ES_5min_raw['Close'].ewm(5).mean().shift()
ES_5min_raw['SMA_50'] = ES_5min_raw['Close'].rolling(50).mean().shift()
ES_5min_raw['SMA_100'] = ES_5min_raw['Close'].rolling(100).mean().shift()
ES_5min_raw['SMA_200'] = ES_5min_raw['Close'].rolling(200).mean().shift()

ES_30min_raw['EMA_9'] = ES_30min_raw['Close'].ewm(5).mean().shift()
ES_30min_raw['SMA_50'] = ES_30min_raw['Close'].rolling(50).mean().shift()
ES_30min_raw['SMA_100'] = ES_30min_raw['Close'].rolling(100).mean().shift()
ES_30min_raw['SMA_200'] = ES_30min_raw['Close'].rolling(200).mean().shift()

ES_1hour_raw['EMA_9'] = ES_1hour_raw['Close'].ewm(5).mean().shift()
ES_1hour_raw['SMA_50'] = ES_1hour_raw['Close'].rolling(50).mean().shift()
ES_1hour_raw['SMA_100'] = ES_1hour_raw['Close'].rolling(100).mean().shift()
ES_1hour_raw['SMA_200'] = ES_1hour_raw['Close'].rolling(200).mean().shift()

fig = go.Figure()
fig.add_trace(go.Scatter(x=ES_1hour_raw.DateTime, y=ES_1hour_raw.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=ES_1hour_raw.DateTime, y=ES_1hour_raw.SMA_50, name='SMA 50'))
fig.add_trace(go.Scatter(x=ES_1hour_raw.DateTime, y=ES_1hour_raw.SMA_100, name='SMA 100'))
fig.add_trace(go.Scatter(x=ES_1hour_raw.DateTime, y=ES_1hour_raw.SMA_200, name='SMA 200'))
fig.add_trace(go.Scatter(x=ES_1hour_raw.DateTime, y=ES_1hour_raw.Close, name='Close', line_color='dimgray', opacity=0.3))
fig.show()

# Nasdaq

NQ_1min_raw['EMA_9'] = NQ_1min_raw['Close'].ewm(5).mean().shift()
NQ_1min_raw['SMA_50'] = NQ_1min_raw['Close'].rolling(50).mean().shift()
NQ_1min_raw['SMA_100'] = NQ_1min_raw['Close'].rolling(100).mean().shift()
NQ_1min_raw['SMA_200'] = NQ_1min_raw['Close'].rolling(200).mean().shift()

NQ_5min_raw['EMA_9'] = NQ_5min_raw['Close'].ewm(5).mean().shift()
NQ_5min_raw['SMA_50'] = NQ_5min_raw['Close'].rolling(50).mean().shift()
NQ_5min_raw['SMA_100'] = NQ_5min_raw['Close'].rolling(100).mean().shift()
NQ_5min_raw['SMA_200'] = NQ_5min_raw['Close'].rolling(200).mean().shift()

NQ_30min_raw['EMA_9'] = NQ_30min_raw['Close'].ewm(5).mean().shift()
NQ_30min_raw['SMA_50'] = NQ_30min_raw['Close'].rolling(50).mean().shift()
NQ_30min_raw['SMA_100'] = NQ_30min_raw['Close'].rolling(100).mean().shift()
NQ_30min_raw['SMA_200'] = NQ_30min_raw['Close'].rolling(200).mean().shift()

NQ_1hour_raw['EMA_9'] = NQ_1hour_raw['Close'].ewm(5).mean().shift()
NQ_1hour_raw['SMA_50'] = NQ_1hour_raw['Close'].rolling(50).mean().shift()
NQ_1hour_raw['SMA_100'] = NQ_1hour_raw['Close'].rolling(100).mean().shift()
NQ_1hour_raw['SMA_200'] = NQ_1hour_raw['Close'].rolling(200).mean().shift()

fig = go.Figure()
fig.add_trace(go.Scatter(x=NQ_1hour_raw.DateTime, y=NQ_1hour_raw.EMA_9, name='EMA 9'))
fig.add_trace(go.Scatter(x=NQ_1hour_raw.DateTime, y=NQ_1hour_raw.SMA_50, name='SMA 50'))
fig.add_trace(go.Scatter(x=NQ_1hour_raw.DateTime, y=NQ_1hour_raw.SMA_100, name='SMA 100'))
fig.add_trace(go.Scatter(x=NQ_1hour_raw.DateTime, y=NQ_1hour_raw.SMA_200, name='SMA 200'))
fig.add_trace(go.Scatter(x=NQ_1hour_raw.DateTime, y=NQ_1hour_raw.Close, name='Close', line_color='dimgray', opacity=0.3))
fig.show()